### Streaming

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages    # Reducer to append the previous communication back to State.
# from langgraph.prebuilt import ToolNode
# from langgraph.prebuilt import tools_condition

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=openai_api_key,
    temperature=0.2,
    streaming=True,
)

In [4]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [3]:
memory = MemorySaver()

In [7]:
def superChatBot(state:State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

In [9]:
# Create graph
builder = StateGraph(State)
builder.add_node("superbot", superChatBot)

builder.add_edge(START, "superbot")
builder.add_edge("superbot", END)

graph = builder.compile(checkpointer=memory)

from IPython.display import Image,display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e) 

Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [ ]:
# Invocation and thread creation

config = {"configurable": {"thread_id": "1"}}
graph.invoke({"messages": "Hello my name is Parag Shah, I like AI"}, config=config)

### Streaming
Methods: .stream() and astream()

These methods are sync and async methods for streaming back results.
Additional parameters in streaming modes for graph state

values : This streams the full state of the graph after each node is called.
updates : This streams updates to the state of the graph after each node is called.

There are 2 modes

#### mode = updates & values

Updates: will print the node / current level messages each time.

Values: will print current and previous (previous msg get appended) level messages every time in the next cycle.

In [10]:
# Create a new thread
config = {"configurable": {"thread_id": "2"}}
for chunk in graph.stream({"messages": "Hello my name is Parag Shah, I like AI"}, config=config,stream_mode="updates"):
    print(chunk)

{'superbot': {'messages': [AIMessage(content="Hello Parag! It's great to meet you. AI is a fascinating field with so many applications and possibilities. What aspects of AI are you most interested in?", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'service_tier': 'default'}, id='run--f5f25cbe-25d6-4bd9-9912-688bc2ea26c4-0')]}}


In [11]:
for chunk in graph.stream({"messages": "Hello my name is Parag Shah, I like AI"}, config=config,stream_mode="values"):
    print(chunk)

{'messages': [HumanMessage(content='Hello my name is Parag Shah, I like AI', additional_kwargs={}, response_metadata={}, id='236964eb-db9e-4200-a5fb-fbf3ab7147e4'), AIMessage(content="Hello Parag! It's great to meet you. AI is a fascinating field with so many applications and possibilities. What aspects of AI are you most interested in?", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'service_tier': 'default'}, id='run--f5f25cbe-25d6-4bd9-9912-688bc2ea26c4-0'), HumanMessage(content='Hello my name is Parag Shah, I like AI', additional_kwargs={}, response_metadata={}, id='2bd1259a-58d1-408a-84ef-0cf42e1d36f3')]}
{'messages': [HumanMessage(content='Hello my name is Parag Shah, I like AI', additional_kwargs={}, response_metadata={}, id='236964eb-db9e-4200-a5fb-fbf3ab7147e4'), AIMessage(content="Hello Parag! It's great to meet you. AI is a fascinating field with so many applications and possib